In [6]:
# Set project directory
import os
import sys


def project_dir():
    notebook_path = %pwd
    repo_name = "sisi"
    repo_folder = notebook_path.split(repo_name)[0]
    return os.path.join(repo_folder, repo_name)


pwd = os.getenv("PWD", project_dir())
os.environ["PWD"] = pwd
sys.path.append(pwd)

In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from Utils.data_utils import *
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt
from pandas import Timestamp
import time
import pickle
import warnings
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)

In [10]:
(train_list, test_list), country_list = load_train_test_data_eu(pwd)

In [11]:
for i in range(len(train_list)):
    train_list[i] = train_list[i].reset_index()
    train_list[i].rename(columns = {'Date':'ds', 'carbon_per_MWh':'y'}, inplace = True)
    train_list[i]['ds'] = train_list[i]['ds'].apply(lambda x : x.replace(tzinfo=None))

In [12]:
for i in range(len(test_list)):
    test_list[i] = test_list[i].reset_index()
    test_list[i].rename(columns = {'Date':'ds', 'carbon_per_MWh':'y'}, inplace = True)
    test_list[i]['ds'] = test_list[i]['ds'].apply(lambda x : x.replace(tzinfo=None))

In [13]:
predictions = []
errors = []
for train, test in zip(train_list, test_list):
    m = Prophet()
    m.fit(train)
    forecast = m.predict(test)
    predictions.append(forecast['yhat'])
    errors.append(mean_absolute_error(test['y'], forecast['yhat']))

Initial log joint probability = -1030.88
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       41563.2      0.288246       5851.09           1           1      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       41792.5     0.0124241        637.94           1           1      236   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       41974.9   0.000744496       1067.82      0.1566      0.1566      350   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399         42031     0.0552338       450.877           1           1      463   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       42088.1    0.00138393       425.993           1           1      580   
    Iter      log prob        ||dx||      ||grad||       alpha  

Initial log joint probability = -213.943
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       48354.1    0.00585712        739.91           1           1      129   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       48576.5     0.0394455        5106.8      0.7907      0.7907      242   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       48642.1   0.000180842       153.881           1           1      358   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       48653.9    0.00388172       669.481           1           1      477   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       48661.1   0.000424021       326.089           1           1      591   
    Iter      log prob        ||dx||      ||grad||       alpha  

Initial log joint probability = -526.474
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       38240.1     0.0253332       933.176           1           1      115   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       38551.5     0.0170876       998.984           1           1      226   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       38764.9     0.0142063       891.084      0.5109           1      341   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       38885.8     0.0430499       1734.61           1           1      448   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     487       38910.1   7.45076e-05       213.888   1.921e-07       0.001      603  LS failed, Hessian reset 
     499       38911.6     0.0197557    

In [14]:
for country, error in zip(country_list, errors):
    print(f"MAE for {state} = {error}")

NameError: name 'state' is not defined

In [ ]:
plt.figure(figsize=(16,16))
plt.plot(train_list[0]['ds'], train_list[0]['y'], color='black')
plt.plot(test_list[0]['ds'], test_list[0]['y'], color='red')
plt.plot(test_list[0]['ds'], predictions[0], color='blue')

In [ ]:
hour_dict_list = []
for i in range(len(test_list)):
    hour_dict = {test_list[i]['ds'][j] : predictions[i][j] for j in range(len(predictions[i]))}
    hour_dict_list.append(hour_dict)

In [ ]:
hour_dict_list[1]

In [ ]:
predictions[1]

In [ ]:
file = open(f'{pwd}/data/predictions/prophet_pred_eu.pkl', 'wb')
pickle.dump(hour_dict_list, file)
file.close()

In [ ]:
temp = pickle.load(open(f'{pwd}/data/predictions/prophet_pred_eu.pkl', 'rb'))

In [ ]:
temp[1]